In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import os

In [2]:
#stock picks from Xialin's model
csv_path = os.path.join('results', 'ML_result.csv')
#fedfunds
ffunds_path = os.path.join('Data', 'FEDFUNDS.csv')
#s&p 500 sectors
sector_path = os.path.join('Data', 'stockssp500sectors.csv')

sector_data_path = os.path.join('results', 'Sector_LinearRegression.csv')

In [3]:
def williams_r(data, lookback=14):
    highh = data['High'].rolling(lookback).max()
    lowl = data['Low'].rolling(lookback).min()
    wr = -100 * ((highh - data['Close']) / (highh - lowl))
    return wr

stocks_df = pd.read_csv(csv_path)
ffunds = pd.read_csv(ffunds_path)
ffunds['DATE'] = pd.to_datetime(ffunds['DATE'])
ffunds.set_index('DATE', inplace=True)
sectors_df = pd.read_csv(sector_path)

print("Sector CSV columns:", sectors_df.columns)
print(sectors_df.head())

stock_to_sector = dict(zip(sectors_df['Stock'], sectors_df['Sector']))

results = []

for index, row in stocks_df.iterrows():
    company = row[0]
  
    if company not in stock_to_sector:
        print(f"No sector found for {company}")
        continue
    
    sector = stock_to_sector[company]

    print(f"Company: {company}, Sector: {sector}")

    company_data = yf.download(company, start='2010-01-01', end='2024-01-01', progress=False)
    sector_data = yf.download(sector, start='2010-01-01', end='2024-01-01', progress=False)

    print(f"Company data head: {company_data.head()}")
    print(f"Sector ({sector}) data head: {sector_data.head()}")

    if company_data.empty or sector_data.empty:
        print(f"Data for {company} or {sector} is empty")
        continue

    monthly_company = company_data.resample('M').mean()
    monthly_sector = sector_data.resample('M').mean()

    if 'Close' not in monthly_company.columns or 'Close' not in monthly_sector.columns:
        print(f"Close column missing in data for {company} or {sector}")
        continue

    monthly_company['Return'] = (monthly_company['Close'].shift(-21) - monthly_company['Close']) / monthly_company['Close']
    monthly_sector['Return'] = (monthly_sector['Close'].shift(-21) - monthly_sector['Close']) / monthly_sector['Close']

    company_data['Williams_%R'] = williams_r(company_data)
    monthly_company['Williams_%R'] = company_data['Williams_%R'].resample('M').last()

    data = pd.DataFrame(index=monthly_company.index)
    data['Company_Return'] = monthly_company['Return']
    data['Sector_Return'] = monthly_sector['Return']
    data['Williams_%R'] = monthly_company['Williams_%R']

    monthly_ffunds = ffunds.resample('M').mean()
    data = data.join(monthly_ffunds['FEDFUNDS'])

    data['Target'] = (data['Company_Return'] > data['Sector_Return']).astype(int)

    data.dropna(inplace=True)

    X = data[['Company_Return', 'Sector_Return', 'FEDFUNDS', 'Williams_%R']]
    y = data['Target']

    random_state = np.random.randint(0, 42)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)

    model = RandomForestClassifier(n_estimators=100, random_state=random_state)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Company: {company}")
    print(f"Accuracy: {accuracy}")
    print(classification_report(y_test, y_pred))

    new_data = X_test.iloc[-1:].copy()
    prediction = model.predict(new_data)
    prediction_text = f"{company} will outperform {sector}" if prediction[0] == 1 else f"{company} will underperform {sector}"
    print("Prediction: ", prediction_text)
    
    # 获取公司的beta值
    company_info = yf.Ticker(company)
    beta = company_info.info.get('beta', None)
    
    results.append([company, sector, accuracy, prediction_text, beta])
    
    print("\n" + "="*50 + "\n")

results_df = pd.DataFrame(results, columns=['Company', 'Sector', 'Accuracy', 'Prediction', 'Beta'])
print(results_df)

Sector CSV columns: Index(['Stock', 'Sector', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')
   Stock     Sector  Unnamed: 2  Unnamed: 3  Unnamed: 4
0  GOOGL  ^SP500-50         NaN         NaN         NaN
1   GOOG  ^SP500-50         NaN         NaN         NaN
2      T  ^SP500-50         NaN         NaN         NaN
3   CHTR  ^SP500-50         NaN         NaN         NaN
4  CMCSA  ^SP500-50         NaN         NaN         NaN
Company: AAPL, Sector: ^SP500-45


/var/folders/n3/cwx8131570q5ljqcv1f22n6c0000gn/T/ipykernel_56752/3912147006.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company = row[0]


Company data head:                 Open      High       Low     Close  Adj Close     Volume
Date                                                                    
2010-01-04  7.622500  7.660714  7.585000  7.643214   6.454506  493729600
2010-01-05  7.664286  7.699643  7.616071  7.656429   6.465663  601904800
2010-01-06  7.656429  7.686786  7.526786  7.534643   6.362820  552160000
2010-01-07  7.562500  7.571429  7.466071  7.520714   6.351057  477131200
2010-01-08  7.510714  7.571429  7.466429  7.570714   6.393281  447610800
Sector (^SP500-45) data head:                   Open        High         Low       Close   Adj Close  \
Date                                                                     
2010-01-04  370.709991  377.500000  370.709991  376.459991  376.459991   
2010-01-05  376.459991  377.279999  373.850006  376.329987  376.329987   
2010-01-06  376.320007  376.839996  372.220001  373.029999  373.029999   
2010-01-07  373.029999  373.029999  369.959991  371.600006  371.600006

/var/folders/n3/cwx8131570q5ljqcv1f22n6c0000gn/T/ipykernel_56752/3912147006.py:41: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_company = company_data.resample('M').mean()
/var/folders/n3/cwx8131570q5ljqcv1f22n6c0000gn/T/ipykernel_56752/3912147006.py:42: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_sector = sector_data.resample('M').mean()
/var/folders/n3/cwx8131570q5ljqcv1f22n6c0000gn/T/ipykernel_56752/3912147006.py:52: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_company['Williams_%R'] = company_data['Williams_%R'].resample('M').last()
/var/folders/n3/cwx8131570q5ljqcv1f22n6c0000gn/T/ipykernel_56752/3912147006.py:59: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_ffunds = ffunds.resample('M').mean()
/var/folders/n3/cwx8131570q5ljqcv1f

Company data head:                  Open       High        Low      Close  Adj Close    Volume
Date                                                                       
2013-01-02  34.919998  35.400002  34.099998  35.119999  21.629183  13767900
2013-01-03  35.000000  35.000000  34.160000  34.830002  21.450581  16739300
2013-01-04  34.619999  34.889999  34.250000  34.389999  21.179594  21372100
2013-01-07  34.150002  35.450001  34.150002  34.459999  21.222708  17897100
2013-01-08  34.290001  34.639999  33.360001  33.709999  20.760809  17863300
Sector (^SP500-35) data head:                   Open        High         Low       Close   Adj Close  \
Date                                                                     
2010-01-04  362.220001  367.339996  362.220001  367.260010  367.260010   
2010-01-05  367.250000  367.250000  363.950012  365.149994  365.149994   
2010-01-06  365.079987  367.130005  365.000000  366.890015  366.890015   
2010-01-07  366.890015  368.519989  365.399994  3

...

Company data head:                  Open       High        Low      Close  Adj Close    Volume
Date                                                                       
2019-06-12  63.500000  67.000000  56.000000  58.000000  58.000000  19449200
2019-06-13  63.860001  69.669998  61.599998  67.559998  67.559998  10923900
2019-06-14  69.779999  70.790001  63.020000  64.160004  64.160004   6264200
2019-06-17  64.750000  70.010002  62.000000  70.010002  70.010002   3247500
2019-06-18  73.000000  78.699997  71.500000  76.500000  76.500000   6612600
Sector (^SP500-45) data head:                   Open        High         Low       Close   Adj Close  \
Date                                                                     
2010-01-04  370.709991  377.500000  370.709991  376.459991  376.459991   
2010-01-05  376.459991  377.279999  373.850006  376.329987  376.329987   
2010-01-06  376.320007  376.839996  372.220001  373.029999  373.029999   
2010-01-07  373.029999  373.029999  369.959991  3

/var/folders/n3/cwx8131570q5ljqcv1f22n6c0000gn/T/ipykernel_56752/3912147006.py:41: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_company = company_data.resample('M').mean()
/var/folders/n3/cwx8131570q5ljqcv1f22n6c0000gn/T/ipykernel_56752/3912147006.py:42: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_sector = sector_data.resample('M').mean()
/var/folders/n3/cwx8131570q5ljqcv1f22n6c0000gn/T/ipykernel_56752/3912147006.py:52: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_company['Williams_%R'] = company_data['Williams_%R'].resample('M').last()
/var/folders/n3/cwx8131570q5ljqcv1f22n6c0000gn/T/ipykernel_56752/3912147006.py:59: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_ffunds = ffunds.resample('M').mean()
/var/folders/n3/cwx8131570q5ljqcv1f

Company data head:                  Open       High        Low      Close  Adj Close   Volume
Date                                                                      
2010-01-04  48.900002  49.180000  48.189999  48.450001  35.008846  5625400
2010-01-05  48.480000  49.130001  47.770000  48.580002  35.102764  7190500
2010-01-06  49.169998  50.090000  49.119999  49.610001  35.847042  8018700
2010-01-07  49.509998  49.509998  48.689999  49.290001  35.615810  7620300
2010-01-08  49.509998  49.790001  48.560001  49.619999  35.854256  5953700
Sector (^SP500-15) data head:                   Open        High         Low       Close   Adj Close  \
Date                                                                     
2010-01-04  199.809998  205.410004  199.809998  205.360001  205.360001   
2010-01-05  205.360001  206.449997  204.860001  206.429993  206.429993   
2010-01-06  206.350006  210.440002  206.350006  209.570007  209.570007   
2010-01-07  209.570007  209.570007  206.610001  208.6100

/var/folders/n3/cwx8131570q5ljqcv1f22n6c0000gn/T/ipykernel_56752/3912147006.py:41: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_company = company_data.resample('M').mean()
/var/folders/n3/cwx8131570q5ljqcv1f22n6c0000gn/T/ipykernel_56752/3912147006.py:42: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_sector = sector_data.resample('M').mean()
/var/folders/n3/cwx8131570q5ljqcv1f22n6c0000gn/T/ipykernel_56752/3912147006.py:52: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_company['Williams_%R'] = company_data['Williams_%R'].resample('M').last()
/var/folders/n3/cwx8131570q5ljqcv1f22n6c0000gn/T/ipykernel_56752/3912147006.py:59: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_ffunds = ffunds.resample('M').mean()




    Company       Sector  Accuracy                         Prediction   Beta
0      AAPL    ^SP500-45  0.900000     AAPL will outperform ^SP500-45  1.239
1      ABBV    ^SP500-35  0.956522     ABBV will outperform ^SP500-35  0.618
2       ABT    ^SP500-35  1.000000      ABT will outperform ^SP500-35  0.722
3       ACN    ^SP500-45  0.866667    ACN will underperform ^SP500-45  1.239
4      ADBE    ^SP500-45  0.933333     ADBE will outperform ^SP500-45  1.289
..      ...          ...       ...                                ...    ...
150    MRNA    ^SP500-35  1.000000   MRNA will underperform ^SP500-35  1.661
151    UBER    ^SP500-20  1.000000     UBER will outperform ^SP500-20  1.331
152     RCL  ^SP500-2550  0.866667  RCL will underperform ^SP500-2550  2.622
153    CRWD    ^SP500-45  1.000000     CRWD will outperform ^SP500-45  1.098
154     NEM    ^SP500-15  0.866667    NEM will underperform ^SP500-15  0.513

[155 rows x 5 columns]


In [4]:
outperform_results = results_df[results_df['Prediction'].str.contains("outperform")]

print(outperform_results)

    Company       Sector  Accuracy                      Prediction   Beta
0      AAPL    ^SP500-45  0.900000  AAPL will outperform ^SP500-45  1.239
1      ABBV    ^SP500-35  0.956522  ABBV will outperform ^SP500-35  0.618
2       ABT    ^SP500-35  1.000000   ABT will outperform ^SP500-35  0.722
4      ADBE    ^SP500-45  0.933333  ADBE will outperform ^SP500-45  1.289
6       ADP    ^SP500-20  0.866667   ADP will outperform ^SP500-20  0.801
..      ...          ...       ...                             ...    ...
142      EL  ^SP500-3010  1.000000  EL will outperform ^SP500-3010  1.002
148    PARA    ^SP500-50  0.966667  PARA will outperform ^SP500-50  1.771
149    SPGI    ^SP500-40  0.900000  SPGI will outperform ^SP500-40  1.176
151    UBER    ^SP500-20  1.000000  UBER will outperform ^SP500-20  1.331
153    CRWD    ^SP500-45  1.000000  CRWD will outperform ^SP500-45  1.098

[84 rows x 5 columns]


In [5]:

df = outperform_results[['Company', 'Beta', 'Sector']]
sectors_data = pd.read_csv(sector_data_path)
df = df.merge(sectors_data[['Sector', 'Coefficient']], left_on='Sector', right_on='Sector', how='left')

df = df.rename(columns={'Company': 'Stock'})

In [6]:
output_path = os.path.join('results', 'portfolio_pick.csv')

results_df.to_csv(output_path, index=False)

print(f"saved to {output_path}")

saved to results/portfolio_pick.csv
